### TODO

1. lambda functions to create quote and order notification messages
2. implement threads for handling live streaming
  - Design json message structure for each handler
  - Implement decoder for json messages for each handler
3. Implement trade job - init, scanner, order
4. Implement user request handler


#### Future
1. Define function to restart a stopped thread
2. Watchdog implementation to resume processes
3. Implementation of user initiated aborts and restart

## Thread functions

In [2]:
from lib.multitasking_lib import *
from lib.kite_helper_lib import *
import sys

## Definition of handler functions

In [9]:
# A thread function to process notifications and tick

trade_lock = Lock() #TODO: make lock per stockname
def trade_job(hash_key):
    pdebug('trade_job: {}'.format(hash_key))
    
    trade_lock.acquire()
    # Step 1.1: Get stock name from the message    
    
    # Step 1.2: Get state for the stock from the redis
    state = conn.hget(hash_key,'state')
    if not state:
        return
    stock = conn.hget(hash_key,'stock')
    freq = conn.hget(hash_key,'freq')
    pdebug("{}: {}: {}".format(hash_key, stock, state ))
    ohlc_df = pd.read_json(conn.hget(hash_key,'ohlc'))
    
    last_processed = ohlc_df.index[-1].strftime('%Y-%m-%d')
    pdebug("{}=>{}".format(last_processed,conn.hget(hash_key,'last_processed')))
    
    if last_processed == conn.hget(hash_key,'last_processed'):   
        trade_lock.release()
        return
    else:
        conn.hset(hash_key,'last_processed',last_processed)
    
    #print('{}:{}'.format(stock,ohlc_df.index[-1]))
    
    # Step 2: Switch to appropriate state machine based on current state
    if state == 'INIT': # State: Init
        # 1: Populate Redis buffer stock+"OHLCBuffer" with historical data
            # Done inside thread handler
        
        # 2: Set state to Scanning
        conn.hset(hash_key,'state','SCANNING')
        pass
    
    elif state == 'SCANNING':  # State: Scanning
        # 1: Run trading algorithm for entering trade
        tradeDecision = False
        
        # 2: If Algo returns Buy: set State to 'Pending Order: Long'
        if tradeDecision:
            conn.hset(hash_key,'state','PO:LONG')
        
        # 3: If Algo returns Sell: set State to 'Pending Order: Short'
        if tradeDecision:
            conn.hset(hash_key,'state','PO:SHORT')
        
        # 4: Update TradeMetaData: Push order details to OrderQueue
        
        pass
    
    elif state == 'PO:LONG': # State: Pending Order: Long
    
        # 1: On Fill: set State to Long
        conn.hset(hash_key,'state','LONG')
        pass
    
    
    elif state == 'PO:SHORT': # State: Pending Order: Short
    
        # 1: On Fill: set State to Short
        conn.hset(hash_key,'state','SHORT')
        pass
    
    
    elif state == 'LONG': # State: Long
    
        # 1: If notification for AutoSquare Off: set state to init
        
        # 2: Else run trading algorithm for square off
        
        # 3: If algo returns square off: then push square off details to OrderQueue, set state to 'Awaiting Square Off'   
        conn.hset(hash_key,'state','SQUAREOFF')
        pass
    
    
    elif state == 'SHORT': # State: Short
    
        # 1: If notification for AutoSquare Off: set state to init
        
        # 2: Else run trading algorithm for square off
        
        # 3: If algo returns square off: then push square off details to OrderQueue, set state to 'Awaiting Square Off'
    
        conn.hset(hash_key,'state','SQUAREOFF')
        pass
        
    elif state == 'SQUAREOFF':  # State: Awaiting Square Off
        
        conn.hset(hash_key,'state','INIT')
        pass
   
        # 1: On Fill notification: set state to Init

    trade_lock.release()


def trade_handler(manager, msg):
    pdebug('trade_handler: {}'.format(msg))
    # Step 1: Blocking call to msgBufferQueue and notificationQueue
    conn.xtrim('msgBufferQueue',maxlen=0, approximate=False)
    conn.xtrim('notificationQueue',maxlen=0, approximate=False)
    while(True):
        msg_q = conn.xread({'msgBufferQueue':'$','notificationQueue':'$'}, block=0, count=100)
        msgs_q = conn.xread({'msgBufferQueue':'0','notificationQueue':'0'}, block=1, count=100)
        conn.xtrim('msgBufferQueue',maxlen=0, approximate=False)
        conn.xtrim('notificationQueue',maxlen=0, approximate=False)
        
        # Step 2: Process notifications: Start a worker thread for each notification
        
        #TODO
        
        # Step 3: Process tick: Start a worker thread for each msg        
        for msg in msgs_q[0][1]:
            pdebug('trade_handler: {}'.format(msg[1]['msg']))
            
            try:
                data = json.loads(msg[1]['msg'])
            except:
                perror("Un-supported message: {} : {}".format(msg, sys.exc_info()[0]))
                break

            for key in data.keys():
                stock = key.split(':')[1]
                exchange = key.split(':')[0]

            hash_key = stock+'_state'
            freq = conn.hget(hash_key,'freq')
            state = conn.hget(hash_key,'state')


            temp_df = msg_to_ohlc(data)
            if state == 'INIT': # State: Init: Load historical data from cache
                # 1: Populate Redis buffer stock+"OHLCBuffer" with historical data
                toDate = (temp_df.index[0] - timedelta(days=1)).strftime('%Y-%m-%d')
                fromDate = (temp_df.index[0] - timedelta(days=no_of_hist_candles)).strftime('%Y-%m-%d')
                ohlc_data = getData(stock, fromDate, toDate, exchange, freq, False, stock)
            else: # Load data from OHLC buffer in hash
                ohlc_data = pd.read_json(conn.hget(hash_key, 'ohlc'))

            ohlc_data = ohlc_data.append(temp_df)
            conn.hset(hash_key,'ohlc',ohlc_data.to_json())

            # Add to OHLCBuffer in hash
            manager.add(stock, trade_job, False, hash_key)
            pdebug(msg[0])

In [10]:
#freedom_init = threadManager("freedom_init", ["freedom_init"], [freedom_init])   

#freedom = threadManager("freedom", ["user_requests_handler", "kite_simulator", "backtest_handler", "trade_handler","order_handler"], 
#                        [user_requests_handler, kite_simulator, backtest_handler, trade_handler, order_handler])

logger.setLevel(logging.INFO)
freedom = threadManager("freedom", [ "kite_simulator", "trade_handler", "order_handler"], 
                        [kite_simulator, trade_handler, order_handler])


04-18 17:34:23:INFO:	NumExpr defaulting to 2 threads.
04-18 17:34:24:INFO:	Using cache: Not downloading data
04-18 17:34:24:INFO:	Using cache: Not downloading data
04-18 17:34:24:INFO:	2019-01-01=>1999-01-01


WIPRO:2019-01-01 00:00:00


04-18 17:34:24:INFO:	Using cache: Not downloading data
04-18 17:34:24:INFO:	2019-01-02=>2019-01-01


WIPRO:2019-01-02 00:00:00


04-18 17:34:24:INFO:	2019-01-03=>2019-01-02


...

04-18 17:36:13:INFO:	2019-03-01=>2019-02-27


TCS:2019-03-01 00:00:00


04-18 17:36:13:INFO:	2019-03-05=>2019-03-01


TCS:2019-03-05 00:00:00


04-18 17:36:13:INFO:	2019-03-05=>2019-03-05
04-18 17:36:13:INFO:	2019-03-05=>2019-03-05
04-18 17:36:13:INFO:	2019-03-05=>2019-03-05
04-18 17:36:13:INFO:	2019-03-05=>2019-03-05
04-18 17:36:13:INFO:	2019-03-05=>2019-03-05
04-18 17:36:13:INFO:	2019-03-05=>2019-03-05
04-18 17:36:13:INFO:	2019-03-05=>2019-03-05
04-18 17:36:13:INFO:	2019-03-05=>2019-03-05


In [11]:

# Unit Test#1
freq='day'
exchange='NSE'
symbol='WIPRO'
fromDate = '2019-01-01'
toDate = '2019-05-03'
stock=symbol

state = 'INIT'
hash_key = stock+'_state'

try:
    all_keys = list(conn.hgetall(hash_key).keys())
    conn.hdel(hash_key,*all_keys)
except:
    pass

#conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

logger.setLevel(logging.INFO)
msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)


In [13]:
notification_despatcher(None, 'stop')

In [14]:
# Unit Test#2
stock = 'TCS'
msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)
notification_despatcher(None, 'msg')

In [8]:
conn.pubsub_channels()

['backtest']

In [8]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

True


['order_handler', 'trade_handler', 'backtest']

In [5]:
# Debug
stock='WIPRO'
#notification_despatcher(None, 'msg')
print(stock)
pd.read_json(conn.hget(stock+'_state','ohlc'))

WIPRO


,close,high,low,open,volume
2018-09-24,250.15,254.14,248.31,248.31,3740868
2018-09-25,246.84,251.80,245.30,250.38,3389131
2018-09-26,240.04,247.33,237.89,246.77,7864825
2018-09-27,239.81,243.35,238.12,242.03,17997289
2018-09-28,243.61,245.60,237.14,239.85,6912444
...,...,...,...,...,...
2019-04-22,288.25,289.00,285.10,285.50,8412054
2019-04-23,291.10,291.85,286.70,289.70,6177784
2019-04-24,293.05,294.30,290.20,292.05,9149069
2019-04-25,295.20,295.95,292.85,293.05,10390816


In [16]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

,close,high,low,open,volume
2019-02-15,2029.70,2052.00,2010.95,2049.00,1930065
2019-02-18,1970.30,2041.95,1962.65,2037.60,2942184
2019-02-19,1904.80,1971.00,1892.00,1970.00,4408554
2019-02-20,1914.75,1933.80,1881.30,1915.00,4645778
2019-02-21,1914.20,1940.00,1897.05,1914.75,4683919
2019-02-22,1925.65,1930.00,1905.10,1917.20,2271955
2019-02-25,1985.15,1990.00,1930.50,1932.50,2934880
2019-02-26,2038.70,2045.15,1972.25,1984.00,6453309
2019-02-27,2058.10,2074.95,2022.00,2040.00,4732082
2019-02-28,1983.45,2071.35,1977.60,2060.00,8454295


## Scratchpad

In [ ]:

 
if False:
    msg_ohlc = {"NSE:WIPRO": {"ohlc": {"date": "2019-1-1", "open": 1896.0, "high": 1910.0, "low": 1885.0, "close": 1902.8, "volume": 1094883.0}}}

    stock='WIPRO'
    state = 'INIT'
    hash_key = stock+'_state'

    try:
        all_keys = list(conn.hgetall(hash_key).keys())
        conn.hdel(hash_key,*all_keys)
    except:
        pass

    conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

    logger.setLevel(logging.INFO)
    trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))

if False:
    msg_ohlc = {"NSE:WIPRO": {"ohlc": {"date": "2019-1-3", "open": 1896.0, "high": 1910.0, "low": 1885.0, "close": 1902.8, "volume": 1094883.0}}}
    trade_job(json.dumps(msg_ohlc))